### **Fundamentos de MPI: Modelos, Tipos de Comunicação e Implementações**

## 1. Modelos de paralelismo em MPI

Em MPI (Message Passing Interface), um programa paralelo é composto por múltiplos processos independentes, cada um identificado por um número (rank). Estes processos executam o mesmo programa e comunicam entre si trocando mensagens.
Existem dois grandes modelos de paralelismo:

**1.1. Decomposição espacial**

A decomposição espacial divide o espaço de dados entre os processos.
Cada processo executa o mesmo código, mas sobre uma parte distinta dos dados.
Este modelo é adequado quando os dados são naturalmente separáveis, como linhas de uma imagem, blocos de uma matriz ou conjuntos de ficheiros.

**Exemplo:**
Num livro de vários capítulos, várias pessoas podem contar as palavras em paralelo — cada pessoa lê um capítulo diferente. No final, os resultados são somados.
Em MPI, cada processo seria responsável pelo seu “capítulo” de dados, e o processo principal recolheria os resultados globais.

**Dependência entre partes dos dados:**
Em alguns casos, os cálculos de uma parte dependem de valores adjacentes que pertencem a outra parte.
Isto acontece, por exemplo, quando se aplica um filtro de média móvel ou uma convolução.

**Exemplo prático:**
Para calcular a média de três pontos de um vetor (x[i-1], x[i], x[i+1]), o processo que detém o último valor do primeiro bloco precisa de x[i+1], pertencente ao processo seguinte.
Neste caso, é necessário trocar linhas ou colunas de fronteira entre processos — as chamadas linhas de halo.
Este mecanismo chama-se halo exchange e permite que os processos vizinhos partilhem apenas a informação mínima indispensável.

**Vantagens:** grande escalabilidade e simplicidade conceptual.
**Desafio:** exige comunicação adicional nas fronteiras entre blocos.

**1.2. Decomposição temporal**

A decomposição temporal divide o processamento em etapas sucessivas.
Cada processo representa uma fase distinta de um fluxo de execução, formando uma cadeia (ou pipeline).

**Exemplo:**
Numa linha de produção, o primeiro trabalhador corta o material, o segundo pinta e o terceiro embala.
Em MPI, o primeiro processo gera dados, o segundo processa e o terceiro grava resultados.

**Vantagens:** útil quando há dependência lógica entre fases que podem sobrepor-se no tempo.
**Desafio:** se uma fase for muito mais lenta, torna-se um gargalo e reduz o paralelismo global.

## 2. Tipos de comunicação em MPI

MPI disponibiliza diferentes mecanismos de comunicação, adequados a cada modelo de paralelismo.

**2.1. Comunicação ponto-a-ponto**

Cada operação de comunicação ponto-a-ponto envolve um emissor e um recetor específicos.
Embora cada par seja independente, vários pares podem comunicar em simultâneo dentro do mesmo programa.

**Principais comandos:**

Send() – envia dados e aguarda a conclusão do envio.

Recv() – espera até receber dados.

Isend() – inicia o envio, mas o processo pode continuar a executar.

Irecv() – inicia a receção, permitindo continuar outras operações enquanto os dados chegam.

Wait() – confirma que uma operação não bloqueante terminou.

**Analogia:**
Enviar e ler cartas.
No envio bloqueante (Send/Recv), o remetente espera que o destinatário abra a carta.
No envio não bloqueante (Isend/Irecv), o remetente coloca a carta no correio e continua a trabalhar; mais tarde confirma que foi entregue.

**2.2. Comunicação coletiva**

As comunicações coletivas envolvem todos os processos de um grupo e são coordenadas automaticamente pela biblioteca MPI.

**Principais comandos:**

Bcast() – envia o mesmo valor a todos os processos.

Scatter() – divide um conjunto de dados e distribui partes diferentes.

Gather() – recolhe resultados de todos e junta-os num só processo.

Reduce() – combina valores de todos os processos (por soma, média, máximo, etc.).

Barrier() – força todos os processos a esperar no mesmo ponto antes de continuar.

**Exemplo de Reduce:**
Se cada processo calcula o número de palavras no seu bloco e se utiliza Reduce com MPI.SUM, o MPI soma automaticamente todos os resultados e entrega a soma total ao processo principal (root).
Se se pretender calcular a média, soma-se primeiro com MPI.SUM e depois divide-se o resultado total pelo número de processos (size).

**2.3. Comunicação com vizinhança (halo exchange)**

Usada quando os dados estão organizados num domínio contínuo (imagem, matriz, grelha) e cada parte depende das fronteiras da parte vizinha.

Funcionamento:

>Cada processo processa o seu bloco principal.

>Antes de cada iteração, envia as linhas ou colunas de fronteira para os vizinhos e recebe as correspondentes linhas de halo.

>Enquanto as mensagens circulam, calcula o interior do bloco.

>Quando as mensagens chegam, calcula as fronteiras com base nos halos recebidos.

**Analogia:**
Várias pessoas pintam uma parede dividida em quadrados.
Para que as cores coincidam nas bordas, cada pessoa mostra uma pequena faixa da sua pintura ao vizinho antes de continuar.

O halo exchange é normalmente implementado com Isend() e Irecv() para permitir que o cálculo do interior decorra em simultâneo com a transferência dos halos.

## 3. Comunicação bloqueante e não bloqueante

**3.1. Bloqueante (Send, Recv)**

O processo espera que a operação termine antes de continuar.

Porquê usar:
É simples e garante ordem previsível das trocas.

Problemas possíveis:

Espera ociosa: o emissor fica parado até o recetor estar pronto.

Deadlock: dois processos podem ficar presos se ambos fizerem Send antes de qualquer Recv.

Como evitar:

Definir uma ordem fixa (por exemplo, pares enviam primeiro, ímpares recebem).

Usar etiquetas (tags) para distinguir mensagens.

Fazer as receções antes dos envios quando possível.

Analogia:
Entregar uma carta em mãos e esperar que o destinatário a leia antes de sair.

**3.2. Não bloqueante (assíncrona) (Isend, Irecv, Wait)**

O processo inicia a comunicação e continua a trabalhar enquanto a mensagem é transmitida.

Porquê usar:
Permite sobrepor comunicação e computação, melhorando o aproveitamento do tempo.
É essencial em trocas de halos e pipelines.

Problemas possíveis:

Esquecimento de Wait: os dados podem não estar prontos quando usados.

Alteração de buffers enquanto a mensagem ainda está a ser enviada.

Como evitar:

Usar Wait ou Waitall para garantir a conclusão das operações.

Não modificar nem reutilizar os buffers de envio até a operação terminar.

Analogia:
Colocar uma carta no correio e continuar a trabalhar, confirmando mais tarde que foi entregue.

## 4. Gargalos (bottlenecks)

Um gargalo ocorre quando um processo ou uma parte do programa concentra demasiado trabalho ou comunicação, limitando o desempenho global.

Exemplo clássico:
Num modelo master–worker, o processo principal (rank 0) distribui tarefas (Send) e recebe resultados (Recv) de todos os outros.
Enquanto o mestre está ocupado a enviar e receber, os workers esperam.
Este atraso chama-se gargalo de comunicação.

Podem existir? Sim, algumas fases seriais são inevitáveis (por exemplo, leitura ou escrita única num ficheiro).
Devem ser evitados? Devem ser minimizados, pois reduzem o ganho de velocidade (speedup).

Como minimizar:

Substituir trocas manuais por comunicações coletivas (Scatter, Gather, Reduce).

Distribuir trabalho de forma equilibrada.

Usar comunicações não bloqueantes (Isend, Irecv) para sobrepor transferência e cálculo.

Dividir tarefas seriais por grupos ou fases paralelas sempre que possível.

Analogia:
Num restaurante, se apenas um empregado servir todas as mesas, o serviço atrasa. Se cada empregado servir uma zona, o trabalho flui em paralelo e o tempo de espera reduz-se.

## 5. Principais comandos MPI e respetivo comportamento

| Comando   | Tipo de comunicação           | Comportamento                                  | Exemplo                              |
| --------- | ----------------------------- | ---------------------------------------------- | ------------------------------------ |
| `Send`    | Ponto-a-ponto, bloqueante     | Envia dados e espera confirmação               | Enviar número do processo 0 para o 1 |
| `Recv`    | Ponto-a-ponto, bloqueante     | Espera e recebe dados de outro processo        | Receber número enviado pelo 0        |
| `Isend`   | Ponto-a-ponto, não bloqueante | Inicia envio e continua execução               | Enviar dados enquanto calcula        |
| `Irecv`   | Ponto-a-ponto, não bloqueante | Inicia receção e prossegue execução            | Receber dados em fundo               |
| `Wait`    | —                             | Aguarda fim de comunicação não bloqueante      | Confirmar entrega ou receção         |
| `Barrier` | Coletiva                      | Todos esperam no mesmo ponto                   | Sincronizar início de uma nova fase  |
| `Bcast`   | Coletiva                      | Envia o mesmo valor a todos                    | Difundir variável comum              |
| `Scatter` | Coletiva                      | Distribui partes de um conjunto de dados       | Enviar subconjuntos a cada processo  |
| `Gather`  | Coletiva                      | Recolhe resultados de todos                    | Juntar resultados num só processo    |
| `Reduce`  | Coletiva                      | Combina resultados (soma, média, máximo, etc.) | Calcular soma total ou média global  |


## 6. Modelos, implementações e exemplos de código

| Código / Exemplo                           | Modelo de paralelismo                       | Tipo de comunicação                                   | Padrão / implementação                    | Descrição                                                                                                                                                                                                                                                                      |
| ------------------------------------------ | ------------------------------------------- | ----------------------------------------------------- | ----------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **ex1_hello_ordenado.py**                  | **Temporal**                                | **Coletiva (`Barrier`)**                              | **Sincronização por turnos**              | Cada processo imprime a sua mensagem apenas no seu turno, aguardando nos mesmos pontos de barreira que os restantes. Demonstra coordenação temporal e execução ordenada.                                                                                                       |
| **ex2_broadcast_matriz.py**                | **Espacial (comum a todos)**                | **Coletiva (`Bcast`)**                                | **Difusão global**                        | O processo *root* cria uma matriz e envia cópias idênticas a todos os outros. Mostra a transmissão um-para-todos de um estado inicial comum.                                                                                                                                   |
| **ex3_scatter_gather_linhas.py**           | **Espacial (distribuição por blocos)**      | **Coletivas (`Scatterv` / `Gatherv` / `Bcast`)**      | **Partição e recomposição**               | A matriz é dividida por linhas; cada processo trata o seu sub-bloco local, depois o *root* recompõe a matriz completa. Ilustra o padrão clássico de paralelismo de dados com partilha estruturada.                                                                             |
| **ex4_reduce_soma_matrizes.py**            | **Espacial (redução global)**               | **Coletiva (`Reduce`)**                               | **Agregação de resultados**               | Cada processo cria uma matriz diferente; o *root* soma todas elemento-a-elemento. Mostra como combinar contributos locais num resultado único (soma ou média).                                                                                                                 |
| **ex5_p2p_anel_sendrecv.py**               | **Espacial (topologia em anel)**            | **Ponto-a-ponto (`Sendrecv`)**                        | **Troca simétrica entre vizinhos**        | Cada processo envia o seu vetor ao vizinho seguinte e recebe do anterior. `Sendrecv` evita *deadlocks* porque o envio e a receção ocorrem num único passo sincronizado.                                                                                                        |
| **ex6_allgather_blocos_pequenos.py**       | **Espacial (partilha total)**               | **Coletiva (`Allgather`)**                            | **Recolha global em todos**               | Cada processo tem um pequeno bloco de dados e, no fim, todos obtêm a concatenação de todos os blocos. Produz o mesmo resultado completo em todos os processos.                                                                                                                 |
| **ex7a_audio_file_per_rank_por_genero.py** | **Espacial (independente / file-per-rank)** | **Baixa comunicação (`bcast` + `gather`)**            | **Paralelismo por tarefas independentes** | Cada processo processa um conjunto diferente de ficheiros áudio, calculando métricas locais (RMS). Apenas o *root* agrega os resultados finais. Exemplo de paralelismo “embaraçosamente paralelo”, típico em análises massivas de ficheiros.                                   |
| **ex7b_imagem_blur3x3_halos.py**           | **Espacial (vizinhança)**                   | **Ponto-a-ponto (`Sendrecv`) + Coletiva (`Gatherv`)** | **Halo exchange (estêncil 3×3)**          | Cada processo processa um bloco contíguo da imagem e troca linhas de fronteira com os vizinhos superior e inferior. O *halo exchange* garante continuidade e elimina descontinuidades nas fronteiras internas. Exemplo completo de paralelismo espacial com dependência local. |

